In [ ]:
import manganite
%load_ext manganite

# Portfolio Selection Optimization
This model is an example of the classic [Markowitz portfolio selection optimization model](https://en.wikipedia.org/wiki/Markowitz_model). We want to find the fraction of the portfolio to invest among a set of stocks that balances risk and return. It is a Quadratic Programming (QP) model with vector and matrix data for returns and risk, respectively. This is best suited to a matrix formulation, so we use the Gurobi Python *matrix* interface. The basic model is fairly simple, so we also solve it parametrically to find the efficient frontier.

**Download the Repository** <br /> 
You can download the repository containing this and other examples by clicking [here](https://github.com/Gurobi/modeling-examples/archive/master.zip). 

**Gurobi License** <br /> 
In order to run this Jupyter Notebook properly, you must have a Gurobi license. If you do not have one, you can request an [evaluation license](https://www.gurobi.com/downloads/request-an-evaluation-license/?utm_source=3PW&utm_medium=OT&utm_campaign=WW-MU-MUI-OR-O_LEA-PR_NO-Q3_FY20_WW_JPME_Lost_Luggage_Distribution_COM_EVAL_GitHub&utm_term=Lost%20Luggage%20Distribution&utm_content=C_JPM) as a *commercial user*, or download a [free license](https://www.gurobi.com/academia/academic-program-and-licenses/?utm_source=3PW&utm_medium=OT&utm_campaign=WW-MU-EDU-OR-O_LEA-PR_NO-Q3_FY20_WW_JPME_Lost_Luggage_Distribution_COM_EVAL_GitHub&utm_term=Lost%20Luggage%20Distribution&utm_content=C_JPM) as an *academic user*.


## Model Formulation
### Parameters

We use the [Greek values](https://en.wikipedia.org/wiki/Greeks_\(finance\)) that are traditional in finance:

- $\delta$: n-element vector measuring the change in price for each stock
- $\sigma$: n x n matrix measuring the covariance among stocks

There is one additional parameter when solving the model parametrically:

- r: target return


### Decision Variables
- $x \ge 0$: n-element vector where each element represents the fraction of the porfolio to invest in each stock

### Objective Function
Minimize the total risk, a convex quadratic function:

\begin{equation}
\min x^t \cdot \sigma \cdot x
\end{equation}

### Constraints

Allocate the entire portfolio: the total investments should be 1.0 (100%), where $e$ is a unit vector (all 1's):

\begin{equation}
e \cdot x = 1
\end{equation}


Return: When we solve the model parametrically for different return values $r$, we add a constraint on the target return:

\begin{equation}
\delta \cdot x = r
\end{equation}

## Python Implementation
### Stock data
Use [yfinance](https://pypi.org/project/yfinance/) library to get the latest 2 years of _actual stock data_ from the 20 most profitable US companies, [according to Wikipedia in April 2021](https://en.wikipedia.org/wiki/List_of_largest_companies_in_the_United_States_by_revenue#List_of_companies_by_profit).
### Dashboard
Use manganite package to create a beautiful dashoard from the jupyter notebook

In [ ]:
import yfinance as yf
import numpy as np

import gurobipy as gp
from gurobipy import GRB
from math import sqrt

import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go



In [ ]:
%%mnn widget --type checkbox --tab "Stock Selector" --header "Apple" --var AAPL
AAPL = True

In [ ]:
%%mnn widget --type checkbox --tab "Stock Selector" --header "Microsoft" --var MSFT
MSFT = True

In [ ]:
%%mnn widget --type checkbox --tab "Stock Selector" --header "JPMorgan" --var JPM
JPM = True

In [ ]:
%%mnn widget --type checkbox --tab "Stock Selector" --header "Google" --var GOOG
GOOG = True

In [ ]:
%%mnn widget --type checkbox --tab "Stock Selector" --header "Bank of America" --var BAC
BAC = True

In [ ]:
%%mnn widget --type checkbox --tab "Stock Selector" --header "Intel" --var INTC
INTC = True

In [ ]:
%%mnn widget --type checkbox --tab "Stock Selector" --header "Wells Fargo" --var WFC
WFC = True

In [ ]:
%%mnn widget --type checkbox --tab "Stock Selector" --header "Meta" --var Meta
Meta = True

In [ ]:
stocks_dict = {'AAPL':AAPL, 'MSFT':MSFT, 'JPM':JPM, 'GOOG':GOOG, 'INTC': INTC, 'BAC':BAC, 'WFC':WFC , 'Meta':Meta}
stocks =[]
for key,value in stocks_dict.items():
    if value:
        stocks.append(key)

data = yf.download(stocks, period='2y')

In [ ]:
%%mnn widget --type plot --var fig_stocks --tab "Stock Selector" --position -1 0 6 --header "Stock prices"

df_closing = data['Close']
fig_stocks = px.line(df_closing, x=df_closing.index, y=df_closing.columns,
            #   hover_data={"date": "|%B %d, %Y"},
              title='Stock prices"')
fig_stocks.update_xaxes(
    dtick="M1",
    tickformat="%b\n%Y",
    ticklabelmode="period")
# fig_stocks.show()

In [ ]:
%%mnn execute --on button "Optimize Portfolio" --returns solution

#calculating greeks
closes = np.transpose(np.array(data.Close)) # matrix of daily closing prices
absdiff = np.diff(closes)                   # change in closing price each day
reldiff = np.divide(absdiff, closes[:,:-1]) # relative change in daily closing price
delta = np.mean(reldiff, axis=1)            # mean price change
sigma = np.cov(reldiff)                     # covariance (standard deviations)
std = np.std(reldiff, axis=1)               # standard deviation
df_plot = pd.DataFrame({'std': std, 'delta': delta, 'stocks':stocks})
print('solving QP model')
# Create an empty model
m = gp.Model('portfolio')

# Add matrix variable for the stocks
x = m.addMVar(len(stocks))

# Objective is to minimize risk (squared).  This is modeled using the
# covariance matrix, which measures the historical correlation between stocks
portfolio_risk = x @ sigma @ x
m.setObjective(portfolio_risk, GRB.MINIMIZE)

# Fix budget with a constraint
m.addConstr(x.sum() == 1, 'budget')

# Verify model formulation
# m.write('portfolio_selection_optimization.lp')

# Optimize model to find the minimum risk portfolio
m.optimize()

minrisk_volatility = sqrt(m.ObjVal)
minrisk_return = delta @ x.X

# Create an expression representing the expected return for the portfolio
portfolio_return = delta @ x
target = m.addConstr(portfolio_return == minrisk_return, 'target')

solution = pd.DataFrame(data=np.append(x.X, [minrisk_volatility, minrisk_return]),
             index=stocks + ['Volatility', 'Expected Return'],
             columns=['Minimum Risk Portfolio'])

# Solve for efficient frontier by varying target return
frontier = np.empty((2,0))
for r in np.linspace(delta.min(), delta.max(), 25):
    target.rhs = r
    m.optimize()
    frontier = np.append(frontier, [[sqrt(m.ObjVal)],[r]], axis=1)



In [ ]:
%%mnn widget --type table --tab "Portfolio" --position 0 0 6 --header "Minimal risk portfolio" --var df_min
df_min = solution

In [ ]:
%%mnn widget --type plot --var fig --tab "Portfolio" --position 1 0 6 --header "Efficient Frontier"

# Plot volatility versus expected return for individual stocks
fig1 = px.scatter(df_plot, x="std", y="delta" ,
                                labels = { "std": "Volatility (standard deviation)", "delta": "Expected Return"}, text="stocks" )
fig1.update_traces(textposition="bottom right")

# Plot volatility versus expected return for minimum risk portfolio

fig2 = px.scatter(x=[minrisk_volatility], y=[minrisk_return],text = ['Minimum\nRisk\nPortfolio'])
fig2.update_traces(textposition="bottom right")
fig  = go.Figure(data=fig1.data + fig2.data)

# Plot efficient frontier

fig.add_trace(go.Scatter(x=frontier[0], y=frontier[1], mode='lines', name='Efficient Frontier'))

# Set x and y labels using update_xaxes and update_yaxes
fig.update_xaxes(title_text="Volatility (standard deviation)")
fig.update_yaxes(title_text="Expected Return")

In [ ]:
# Using matplot insted of plotly
# %%mnn widget --type plot --var fig --tab "Portfolio" --position 2 0 6 --header "Efficient Frontier"


# fig, ax = plt.subplots(figsize=(10,8))

# # Plot volatility versus expected return for individual stocks
# ax.scatter(x=std, y=delta,
#            color='Blue', label='Individual Stocks')
# for i, stock in enumerate(stocks):
#     ax.annotate(stock, (std[i], delta[i]))

# # Plot volatility versus expected return for minimum risk portfolio
# ax.scatter(x=minrisk_volatility, y=minrisk_return, color='DarkGreen')
# ax.annotate('Minimum\nRisk\nPortfolio', (minrisk_volatility, minrisk_return),
#             horizontalalignment='right')

# # Plot efficient frontier
# ax.plot(frontier[0], frontier[1], label='Efficient Frontier', color='DarkGreen')

# # Format and display the final plot
# ax.axis([frontier[0].min()*0.7, frontier[0].max()*1.3, delta.min()*1.2, delta.max()*1.2])
# ax.set_xlabel('Volatility (standard deviation)')
# ax.set_ylabel('Expected Return')
# ax.legend()
# ax.grid()
# plt.show()